Proyecto E - Visor

# Exploración y preparación de datos (Calidad de datos)

## Importar librerías

In [46]:
import sys, platform
print("PY:", sys.executable)
print("VER:", platform.python_version())

PY: C:\Users\danig\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
VER: 3.12.10


In [37]:
%pip install --upgrade openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\danig\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import openpyxl

## Cargar los datos

In [39]:
landis0_path = "C:\\Users\\danig\\OneDrive\\Documentos\\GitHub\\E-Visor\\data\\etsmartmeter_JULIO_2025-07-01_a_2025-07-31_1_hora (1).xlsx"
landis1_path = "C:\\Users\\danig\\OneDrive\\Documentos\\GitHub\\E-Visor\\data\\etsmartmeter_AGOSTO_2025-08-01_a_2025-08-31_1_hora.xlsx"
landis2_path = "C:\\Users\\danig\\OneDrive\\Documentos\\GitHub\\E-Visor\\data\\etsmartmeter_SEPTIEMBR_2025-09-01_a_2025-09-30_1_hora.xlsx"
landis3_path = "C:\\Users\\danig\\OneDrive\\Documentos\\GitHub\\E-Visor\\data\\etsmartmeter_OCTUBRE_2025-10-01_a_2025-10-19_1_hora.xlsx"
df0 = pd.read_excel(landis0_path)
df1 = pd.read_excel(landis1_path)
df2 = pd.read_excel(landis2_path)
df3 = pd.read_excel(landis3_path)
df1.head()

,time_index_colombia,entity_id,activeenergyimport,relativethdvoltage,totalpowerfactor,activepower,reactiveenergyimport,reactiveenergyexport,relativethdcurrent,reactivepower
0,2025-08-01 00:00:00,SmartMeter_SM_B10_ARQ,77411230.0,1.731,0.92,7687.2,77982170.0,699774.0,4.145,2778.0
1,2025-08-01 01:00:00,SmartMeter_SM_B10_ARQ,77418660.0,1.533,0.88,6876.7,77985580.0,699774.0,3.756,3505.0
2,2025-08-01 02:00:00,SmartMeter_SM_ECOVILLA,6979993.5,13.673,0.81,329.4,297412.6,4504210.0,4.441,215.0
3,2025-08-01 03:00:00,SmartMeter_SM_ECOVILLA,6980340.0,13.463,0.83,352.2,297412.6,4504427.0,4.983,217.0
4,2025-08-01 04:00:00,SmartMeter_SM_ECOVILLA,6980678.5,13.240,0.83,354.7,297412.6,4504643.0,4.979,216.0


In [40]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   time_index_colombia   744 non-null    datetime64[ns]
 1   entity_id             626 non-null    object        
 2   activeenergyimport    626 non-null    float64       
 3   relativethdvoltage    626 non-null    float64       
 4   totalpowerfactor      626 non-null    float64       
 5   activepower           626 non-null    float64       
 6   reactiveenergyimport  626 non-null    float64       
 7   reactiveenergyexport  626 non-null    float64       
 8   relativethdcurrent    626 non-null    float64       
 9   reactivepower         626 non-null    float64       
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 58.3+ KB


## Integración de datasets

In [41]:
import pandas as pd

dfs = [df0, df1, df2, df3]  # todos tienen las columnas iguales
df = pd.concat(dfs, ignore_index=True)
#quita duplicados exactos (mismo medidor y timestamp)
if {'entity_id','time_index_colombia'}.issubset(df.columns):
    #ver los posibles duplicados, se asumen como duplicados los que tienen mismo 'entity_id' y 'time_index_colombia'!
    duplicados = df[df.duplicated(subset=['entity_id','time_index_colombia'], keep=False)]
    print(f"Duplicados con mismo 'entity_id' y 'time_index_colombia': {len(duplicados)}")
    df = df.drop_duplicates(subset=['entity_id','time_index_colombia'])


Duplicados con mismo 'entity_id' y 'time_index_colombia': 0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2663 entries, 0 to 2662
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   time_index_colombia   2663 non-null   datetime64[ns]
 1   entity_id             2387 non-null   object        
 2   activeenergyimport    2382 non-null   float64       
 3   relativethdvoltage    2382 non-null   float64       
 4   totalpowerfactor      2382 non-null   float64       
 5   activepower           2382 non-null   float64       
 6   reactiveenergyimport  2382 non-null   float64       
 7   reactiveenergyexport  2382 non-null   float64       
 8   relativethdcurrent    2382 non-null   float64       
 9   reactivepower         2382 non-null   float64       
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 208.2+ KB


In [43]:
#Códigos de bloques identificados en el dataset
df['entity_id'].unique()
#Validar que no haya duplicados ni bloques faltantes!

array([nan, 'SmartMeter_SM_B12_DERE', 'SmartMeter_SM_B10_ARQ',
       'SmartMeter_SM_B9_SFA2', 'SmartMeter_SM_B3_RECT',
       'SmartMeter_SM_ECOVILLA', 'SmartMeter_SM_B5_BACH',
       'SmartMeter_SM_B8_AA', 'SmartMeter_SM_B8_CPA',
       'SmartMeter_SM_B4_PRIM', 'SmartMeter_SM_B15_BIBL',
       'SmartMeter_SM_B8_LABS', 'SmartMeter_SM_B7_CTIC',
       'SmartMeter_SM_B9_SFA1', 'SmartMeter_SM_B17_POLI',
       'SmartMeter_SM_B18_PARQ'], dtype=object)

In [44]:
#Formato de fecha y hora de la variable, y zona horaria de 'time_index_colombia'
df['time_index_colombia'] = pd.to_datetime(df['time_index_colombia'], errors='coerce', utc=True)
df['time_index_colombia'] = df['time_index_colombia'].dt.tz_convert('America/Bogota')

In [47]:
df['time_index_colombia'].head()

0   2025-06-30 19:00:00-05:00
1   2025-06-30 20:00:00-05:00
2   2025-06-30 21:00:00-05:00
3   2025-06-30 22:00:00-05:00
4   2025-06-30 23:00:00-05:00
Name: time_index_colombia, dtype: datetime64[ns, America/Bogota]

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2663 entries, 0 to 2662
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype                         
---  ------                --------------  -----                         
 0   time_index_colombia   2663 non-null   datetime64[ns, America/Bogota]
 1   entity_id             2387 non-null   object                        
 2   activeenergyimport    2382 non-null   float64                       
 3   relativethdvoltage    2382 non-null   float64                       
 4   totalpowerfactor      2382 non-null   float64                       
 5   activepower           2382 non-null   float64                       
 6   reactiveenergyimport  2382 non-null   float64                       
 7   reactiveenergyexport  2382 non-null   float64                       
 8   relativethdcurrent    2382 non-null   float64                       
 9   reactivepower         2382 non-null   float64                       
dtype